# IBM Watson ML - Electric Motor Temperature Prediction
### Training and Deploying the model on IBM Cloud

## Step 1: Install IBM Watson Machine Learning SDK

In [ ]:
!pip install ibm-watson-machine-learning --quiet
print('IBM WML SDK installed!')

## Step 2: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import pickle
from ibm_watson_machine_learning import APIClient

print('Libraries imported!')

## Step 3: IBM Watson ML Credentials

In [ ]:
# Replace with your IBM Cloud credentials
wml_credentials = {
    "apikey": "YOUR_IBM_API_KEY",
    "url":    "https://us-south.ml.cloud.ibm.com"
}

client = APIClient(wml_credentials)
print('Connected to IBM Watson ML!')

## Step 4: Load and Prepare Data

In [ ]:
df = pd.read_csv('../pmsm_temperature_data.csv')
if 'profile_id' in df.columns:
    df = df.drop('profile_id', axis=1)

target   = 'pm'
features = [c for c in df.columns if c != target]

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler       = StandardScaler()
X_train_sc   = scaler.fit_transform(X_train)
X_test_sc    = scaler.transform(X_test)

print('Data ready! Train:', X_train.shape, '| Test:', X_test.shape)

## Step 5: Train Model

In [ ]:
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_sc, y_train)

y_pred = model.predict(X_test_sc)
print(f'R² Score : {r2_score(y_test, y_pred):.4f}')
print(f'RMSE     : {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}')

## Step 6: Save and Deploy to IBM Watson ML

In [ ]:
# Save model locally
with open('model.save', 'wb') as f:
    pickle.dump(model, f)
with open('transform.save', 'wb') as f:
    pickle.dump(scaler, f)

print('Model saved locally!')

In [ ]:
# Set deployment space
# client.set.default_space('YOUR_SPACE_ID')

# Store model on IBM WML
model_props = {
    client.repository.ModelMetaNames.NAME: 'MotorTempPredictor',
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_1.0',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:
        client.software_specifications.get_id_by_name('runtime-22.2-py3.10')
}

# published_model = client.repository.store_model(model=model, meta_props=model_props)
# model_uid = client.repository.get_model_id(published_model)
# print('Model stored! UID:', model_uid)

print('(Uncomment above lines and add your Space ID to deploy on IBM Cloud)')

In [ ]:
# Create deployment
# deploy_props = {
#     client.deployments.ConfigurationMetaNames.NAME: 'MotorTemp-Deployment',
#     client.deployments.ConfigurationMetaNames.ONLINE: {}
# }
# deployment = client.deployments.create(model_uid, meta_props=deploy_props)
# scoring_url = client.deployments.get_scoring_href(deployment)
# print('Scoring URL:', scoring_url)

print('Deployment step - update app.py IBM_ENDPOINT with the scoring URL above.')